In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

Define the graph G

In [2]:
class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = list()
        self.edges_out = defaultdict(list)
        self.edges_in = defaultdict(list)
        self.distance = dict()
        self.f_dep_t = dict()
        self.f_park_t = dict()

    def add_node(self, value, feasible_parking_time):
        self.nodes.add(value)
        self.f_park_t[value] = feasible_parking_time

    def add_edge(self, from_node, to_node, distance, feasible_dep_time):
        self.edges.append((from_node, to_node))
        self.edges_out[from_node].append(to_node)
        self.edges_in[to_node].append(from_node)
        self.distance[(from_node, to_node)] = distance
        self.f_dep_t[(from_node, to_node)] = feasible_dep_time

In [559]:
#Knoten der Beispielinstanz
g = Graph()
parking_times = [[0, float('inf')], [18,30], [35, 45], [64,75], [35,50], [64, 75], [0, float('inf')]]
for i in range(1, 9):
    g.add_node(i, parking_times[i-1])

IndexError: list index out of range

In [560]:
#Kanten der Beispielinstanz
g.add_edge(1, 2, 10, [10, 15])
g.add_edge(1, 3, 20, [10, 25])
g.add_edge(1, 5, 25, [15, 25])
g.add_edge(2, 4, 35, [28, 35])
g.add_edge(2, 6, 35, [26, 34])
g.add_edge(2, 7, 30, [32, 60])
g.add_edge(3, 2, 15, [35, 50])
g.add_edge(3, 4, 20, [43, 50])
g.add_edge(4, 7, 30, [70, 99])
g.add_edge(5, 2, 20, [30, 80])
g.add_edge(5, 6, 25, [48, 55])
g.add_edge(6, 7, 10, [72, 80])
g.add_edge(7, 8, 0, [0, 0])

In [561]:
for i in g.nodes:
    print(g.f_park_t[i])

[0, inf]
[18, 30]
[35, 45]
[64, 75]
[35, 50]
[64, 75]
[0, inf]


Write functions

i = 1
j = 2
fij =0
test = check_for_feasibility(i, j, 15)
if test is not False: # here we avoid that zero satisfies a false check
    print("feasible earliest arrival time at node {} at time {} with parking time {}".format(i, fij, test))

In [578]:
opt_is

{(2, 4): 1,
 (2, 6): 1,
 (2, 7): 1,
 (3, 2): 1,
 (3, 4): 1,
 (4, 7): 2,
 (5, 2): 1,
 (5, 6): 1,
 (6, 7): 2,
 (7, 8): 6}

In [577]:
get_route_for_arc((1,2), optimal_i=opt_is)

[]

In [576]:
def get_route_for_arc(arc: tuple, optimal_i: dict):
    check_route_for = arc
    #route = [check_route_for]
    route = list()
    i = check_route_for[0]
    j = check_route_for[1]
    try:
        while True:
            curr_arc = (i,j)
            i = optimal_i[curr_arc]
            j = curr_arc[0]
            route.append((i,j))
            if i == 1:
                break
        return [i for i in reversed(route)] 
    
    except:
        return []
    

In [565]:
check_for_feasibility(1,2,0, opt_is, earliest_feasible_arrival_times)

Check feasibility for usage of arc ((1, 2)) for earliest feasible time 0
feasible parking times for 1 when moving to 2: [10, 11, 12, 13, 14, 15]
1st cond satisfied
du you go here bro? Then why return none? : 10


10

In [566]:
do_recusion(1, 2, 0, opt_is, earliest_feasible_arrival_times)

so lets do the recursion
[]
MAAAAAAAAAAAAAAN das sollte doch gehen=?!?!?!?!?!?!


inf

In [567]:
def do_recusion(i, j, earliest_feasible_at, opt_is, earliest_feasible_arrival_times):
    # specify time window for parking
    print("so lets do the recursion")
    alpha_i = g.f_park_t[i][0]
    beta_i = g.f_park_t[i][1]
    
    # specify time window for traveling the arc
    gamma_ij = g.f_dep_t[(i,j)][0]
    delta_ij = g.f_dep_t[(i,j)][1]

    previous_arcs = get_route_for_arc((i,j), opt_is)
    print(previous_arcs)
    if len(previous_arcs) == 0:
        print("MAAAAAAAAAAAAAAN das sollte doch gehen=?!?!?!?!?!?!")
        return float('inf')
    else:
        # this time is missing on node j in order to be able to park or directly move to k
        time_gap = min([alpha_i - earliest_feasible_at, gamma_ij - earliest_feasible_at])
        if time_gap < 0:
            time_gap = max([alpha_i - earliest_feasible_at, gamma_ij - earliest_feasible_at])
        print("time_gap", time_gap)

        curr_prev_arc = previous_arcs[-1]
        print(curr_prev_arc)
        f_i1_i2 = earliest_feasible_arrival_times[curr_prev_arc]
        new_p_i = check_for_feasibility(*curr_prev_arc, f_i1_i2, opt_is, earliest_feasible_arrival_times, add_park_time=time_gap)
        new_earliest_feasible_at = f_i1_i2 + new_p_i + g.distance[curr_prev_arc]
        print("new f({}): ".format((i,j)), new_earliest_feasible_at, "new_p{}: ".format(i), new_p_i)
        earliest_feasible_arrival_times[i,j] = new_earliest_feasible_at
        parking[curr_prev_arc] = new_p_i
        return check_for_feasibility(i,j,new_earliest_feasible_at, opt_is, earliest_feasible_arrival_times)
     

In [568]:
def check_for_feasibility(i, j, earliest_feasible_at, opt_is, earliest_feasible_arrival_times, add_park_time=None):
    print("Check feasibility for usage of arc ({}) for earliest feasible time {}".format((i,j), earliest_feasible_at))
    if earliest_feasible_at == float('inf'):
        print("HUHUHUU")
        return False
    # specify time window for parking
    alpha_i = g.f_park_t[i][0]
    beta_i = g.f_park_t[i][1]
    
    # specify time window for traveling the arc
    gamma_ij = g.f_dep_t[(i,j)][0]
    delta_ij = g.f_dep_t[(i,j)][1]
    
    # check first equation:
    if alpha_i <= delta_ij:
        #pass
        max_p_i = min(beta_i - earliest_feasible_at, delta_ij - earliest_feasible_at)
        p_i = []
        for feas_p_i in range(max_p_i+1):
            if (alpha_i <= earliest_feasible_at <= beta_i) and (gamma_ij <= earliest_feasible_at + feas_p_i <= delta_ij):
                p_i.append(feas_p_i)

        print("feasible parking times for {} when moving to {}: {}".format(i,j,p_i))
        if len(p_i) > 0:
            print("1st cond satisfied")
            if add_park_time:
                print("want to add additional parking time to node 1: ", add_park_time)
                print("This sums up to ", p_i[0] + add_park_time)
                if p_i[0] + add_park_time < max_p_i:
                    return p_i[0] + add_park_time
                else: 
                    print("NOOOOOOOOOO, max pi is only: ", max_p_i)
                    # So when we move here we set f(2,7) to infinity, but what we really want to do is set
                    # f(1,2) to infinit and with this new insight calculate a new f(2,7): How can we proceed here?
                    #earliest_feasible_arrival_times[1,2] = float('inf')
                    return float('inf')
            else:
                print("du you go here bro? Then why return none? :", p_i[0])
                return p_i[0]
        
    #max_p_i = min(beta_i - earliest_feasible_at, delta_ij - earliest_feasible_at)
    #if (alpha_i <= earliest_feasible_at <= beta_i) and (gamma_ij <= earliest_feasible_at + max_p_i <= delta_ij):
    #    return max_p_i
    # check second equation
    if (beta_i - earliest_feasible_at <= 0) and (gamma_ij <= earliest_feasible_at <= delta_ij):
        print("2nd cond satisfied")
        p_i = 0
        return p_i
    # check third equation
    elif (earliest_feasible_at < gamma_ij <= alpha_i) or (earliest_feasible_at < alpha_i < gamma_ij < beta_i
                                                        ) or earliest_feasible_at < beta_i < gamma_ij:
        a =  do_recusion(i, j, earliest_feasible_at, opt_is, earliest_feasible_arrival_times)
        print("recursion is done, result is: ", a)
        return a
    
    # check fourth equation
    elif gamma_ij < earliest_feasible_at < alpha_i <= delta_ij:
        print("4th cond satisfied")
        p_i = 0
        return [p_i]
    
    # check fifth equation
    elif delta_ij < earliest_feasible_at:
        print("5th cond satisfied")
        return [float('inf')]
    
    else:
        raise
        print(Warning("Some cases are not considered in feasibility check"))

In [569]:
earliest_feasible_arrival_times = dict()
sink = min(g.nodes)
terminal = max(g.nodes)
opt_is = dict()
parking = defaultdict(list)
for k in range(sink, terminal+2):
    if k == 1:
        arcs_from_sink = [jk[1] for jk in g.edges if jk[0] == 1]
        for l in arcs_from_sink:
            earliest_feasible_arrival_times[k, l] = 0
            tour.append((k,l))
    else:
        print("The outer loops current iteration is: ", k)
        
        arcs_to_k = [jk[0] for jk in g.edges if jk[1] == k]
        print("The following nodes move into {}: {}".format(k, arcs_to_k))
        for j in arcs_to_k:
            print("The iteration over j is currently at: ", j)
            if j == 1:
                continue
            pot_earliest_feasible_arrival_times = []
            arcs_to_j = [ij[0] for ij in g.edges if ij[1] == j]
            print("The following nodes move into {}: {}".format(j, arcs_to_j))
            for i in arcs_to_j:
                f_ij = earliest_feasible_arrival_times[i,j]
                print("f({}, {}) is: {}".format(i, j, f_ij))
                p_i = check_for_feasibility(i, j, f_ij, opt_is, earliest_feasible_arrival_times)
                f_ij = earliest_feasible_arrival_times[i,j]
                print("p_{}:".format(i),p_i)
                print("current earliest_feasible_arrival_times:", earliest_feasible_arrival_times)
                t_ij = g.distance[i,j]
                parking[i,j] = p_i
                if p_i is not False and type(p_i) == int:
                    print("f_{}; p_{}; t_{}: ".format((i,j), i, (i,j)), f_ij, "; ", p_i, "; ", t_ij)
                    pot_earliest_feasible_arrival_times.append(f_ij + p_i + t_ij)
                else:
                    pot_earliest_feasible_arrival_times.append(float('inf'))
                print("pot_earliest_feasible_arrival_times", pot_earliest_feasible_arrival_times)
            earliest_feasible_arrival_times[j,k] = min(pot_earliest_feasible_arrival_times)
            print("best i", pot_earliest_feasible_arrival_times.index(min(pot_earliest_feasible_arrival_times)))
            opt_is[j,k] = arcs_to_j[pot_earliest_feasible_arrival_times.index(min(pot_earliest_feasible_arrival_times))]
        #tour.append((j,k))

The outer loops current iteration is:  2
The following nodes move into 2: [1, 3, 5]
The iteration over j is currently at:  1
The iteration over j is currently at:  3
The following nodes move into 3: [1]
f(1, 3) is: 0
Check feasibility for usage of arc ((1, 3)) for earliest feasible time 0
feasible parking times for 1 when moving to 3: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
1st cond satisfied
du you go here bro? Then why return none? : 10
p_1: 10
current earliest_feasible_arrival_times: {(1, 2): 0, (1, 3): 0, (1, 5): 0}
f_(1, 3); p_1; t_(1, 3):  0 ;  10 ;  20
pot_earliest_feasible_arrival_times [30]
best i 0
The iteration over j is currently at:  5
The following nodes move into 5: [1]
f(1, 5) is: 0
Check feasibility for usage of arc ((1, 5)) for earliest feasible time 0
feasible parking times for 1 when moving to 5: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
1st cond satisfied
du you go here bro? Then why return none? : 15
p_1: 15
current earliest_feasible_ar

In [570]:
earliest_feasible_arrival_times

{(1, 2): 0,
 (1, 3): 0,
 (1, 5): 0,
 (2, 4): 20,
 (2, 6): 20,
 (2, 7): inf,
 (3, 2): 35,
 (3, 4): 35,
 (4, 7): 64,
 (5, 2): 40,
 (5, 6): 40,
 (6, 7): 64,
 (7, 8): 82}

In [571]:
get_route_for_arc((7,8), opt_is)

[(1, 2), (2, 6), (6, 7)]

In [572]:
parking

defaultdict(list,
            {(1, 2): inf,
             (1, 3): 15,
             (1, 5): 15,
             (2, 4): 9,
             (2, 6): 9,
             (2, 7): False,
             (3, 2): 0,
             (3, 4): 8,
             (4, 7): 6,
             (5, 2): 0,
             (5, 6): 8,
             (6, 7): 8})

In [515]:
check_for_feasibility(3,2,30,opt_is, earliest_feasible_arrival_times)

Check feasibility for usage of arc ((3, 2)) for earliest feasible time 30
feasible parking times for 3 when moving to 2: []
so lets do the recursion
[(1, 3)]
time_gap 5
(1, 3)
Check feasibility for usage of arc ((1, 3)) for earliest feasible time 0
feasible parking times for 1 when moving to 3: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
1st cond satisfied
want to add additional parking time to node 1:  5
This sums up to  15
new f((3, 2)):  35 new_p3:  15
Check feasibility for usage of arc ((3, 2)) for earliest feasible time 35
feasible parking times for 3 when moving to 2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
1st cond satisfied
du you go here bro? Then why return none? : 0
recursion is done, result is:  0


0

In [516]:
earliest_feasible_arrival_times

{(1, 2): 0,
 (1, 3): 0,
 (1, 5): 0,
 (2, 4): 20,
 (2, 6): 20,
 (2, 7): inf,
 (3, 2): 35,
 (3, 4): 35,
 (4, 7): 64,
 (5, 2): 40,
 (5, 6): 40,
 (6, 7): 64,
 (7, 8): 82}

In [399]:
earliest_feasible_arrival_times

{(1, 2): 0,
 (1, 3): 0,
 (1, 5): 0,
 (2, 4): 25,
 (2, 6): 25,
 (2, 7): 25,
 (3, 2): 45,
 (3, 4): 45,
 (4, 7): 65,
 (5, 2): 50,
 (5, 6): 50,
 (6, 7): 65,
 (7, 8): 85}

In [400]:
get_route_for_arc((7,8), opt_is)

[(1, 2), (2, 6), (6, 7)]

### old solution

In [15]:
earliest_feasible_arrival_times = dict()
sink = min(g.nodes)
terminal = max(g.nodes)

for k in range(sink, terminal+1):
    if k == 1:
        arcs_from_sink = [jk[1] for jk in g.edges if jk[0] == 1]
        for l in arcs_from_sink:
            earliest_feasible_arrival_times[k, l] = 0
    else:
        print("The outer loops current iteration is: ", k)
        arcs_to_k = [jk[0] for jk in g.edges if jk[1] == k]
        print("The following nodes move into {}: {}".format(k, arcs_to_k))
        for j in arcs_to_k:
            print("The iteration over j is currently at: ", j)
            if j == 1:
                continue
            pot_earliest_feasible_arrival_times = []
            arcs_to_j = [ij[0] for ij in g.edges if ij[1] == j]
            print("The following nodes move into {}: {}".format(j, arcs_to_j))
            for i in arcs_to_j:
                f_ij = earliest_feasible_arrival_times[i,j]
                p_i = check_for_feasibility(i, j, f_ij)
                t_ij = g.distance[i,j]
                if p_i is not False:
                    pot_earliest_feasible_arrival_times.append(f_ij + p_i + t_ij)
                else:
                    pot_earliest_feasible_arrival_times.append(float('inf'))
            earliest_feasible_arrival_times[j,k] = min(pot_earliest_feasible_arrival_times)

The outer loops current iteration is:  2
The following nodes move into 2: [1, 3, 5]
The iteration over j is currently at:  1
The iteration over j is currently at:  3
The following nodes move into 3: [1]
1st cond satisfied
The iteration over j is currently at:  5
The following nodes move into 5: [1]
1st cond satisfied
The outer loops current iteration is:  3
The following nodes move into 3: [1]
The iteration over j is currently at:  1
The outer loops current iteration is:  4
The following nodes move into 4: [2, 3]
The iteration over j is currently at:  2
The following nodes move into 2: [1, 3, 5]
1st cond satisfied
1st cond satisfied
2nd cond satisfied
The iteration over j is currently at:  3
The following nodes move into 3: [1]
1st cond satisfied
The outer loops current iteration is:  5
The following nodes move into 5: [1]
The iteration over j is currently at:  1
The outer loops current iteration is:  6
The following nodes move into 6: [2, 5]
The iteration over j is currently at:  2
Th

In [ ]:
def check_for_feasibility(i, j, earliest_feasible_at):
    print("Check feasibility for usage of arc ({}) for earliest feasible time {}".format((i,j), earliest_feasible_at))
    
    # specify time window for parking
    alpha_i = g.f_park_t[i][0]
    beta_i = g.f_park_t[i][1]
    
    # specify time window for traveling the arc
    gamma_ij = g.f_dep_t[(i,j)][0]
    delta_ij = g.f_dep_t[(i,j)][1]
    
    # check first equation:
    if alpha_i <= gamma_ij:
        p_i = kackscheiss(alpha_i, beta_i, earliest_feasible_at, gamma_ij, delta_ij)
        if p_i is not False:
            return p_i
        else:
            pass
    # check second equation
    if (beta_i - earliest_feasible_at <= 0) and (gamma_ij <= earliest_feasible_at <= delta_ij):
        print("2nd cond satisfied")
        p_i = 0
        return p_i
    # check third equation
    elif (earliest_feasible_at < gamma_ij < alpha_i) or (earliest_feasible_at < alpha_i < gamma_ij < beta_i
                                                        ) or earliest_feasible_at < alpha_i < beta_i < gamma_ij:
        print("3rd cond satisfied")
        return False
    
    # check fourth equation
    elif gamma_ij < earliest_feasible_at < alpha_i <= delta_ij:
        print("4th cond satisfied")
        p_i = 0
        return p_i
    
    # check fifth equation
    elif delta_ij < earliest_feasible_at:
        print("5th cond satisfied")
        return False
    
    else:
        print(Warning("Some cases are not considered in feasibility check"))
        return 123